In [46]:
import numpy as np
import pandas as pd
import nltk

pd.set_option('display.max_columns', 500)

# Przetwarzanie zbioru danych

Wczytanie zbioru danych

In [47]:
df = pd.read_csv("reviews_train.csv")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,summary,unixReviewTime,reviewTime,score
0,A35C43YE9HU9CN,B0064X7B4A,Joan Miller,"[0, 0]",I have decided not to play this game. I can't...,Friends,1396396800,"04 2, 2014",1.0
1,AHFS8CGWWXB5B,B00H1P4V3E,WASH ST. GAMER,"[3, 4]",The Amazon Appstore free app of the day for Ju...,"Amazon Makes This ""Longest Spring Ever"" for Fi...",1402272000,"06 9, 2014",2.0
2,A3EW8OTQ90NVHM,B00CLVW82O,Kindle Customer,"[0, 4]",this game was so mush fun I wish I could play ...,best,1368921600,"05 19, 2013",5.0
3,AJ3GHFJY1IUTD,B007T9WVKM,BrawlMaster4,"[0, 2]","Its pretty fun and very good looking, but you...",Fun Game,1350172800,"10 14, 2012",5.0
4,A3JJGBS4EL603S,B00J206J5E,"K. Wilson ""thesupe""","[0, 0]",good graphics; immersive storyline; hard to st...,great game!,1396915200,"04 8, 2014",5.0


Wstępne przetwarzanie danych

In [48]:
import json
from sklearn.preprocessing import LabelEncoder

# Wczytanie zmiana typu kolumny na taki użyteczny dla klasyfikatora.
# for i, val in df.iterrows():
#     # val['helpful'] = json.loads(val['helpful'])
#     df.iloc[i]['helpful']= json.loads(val['helpful'])

# Odrzucanie niepełnych wierszy, ponieważ są one nieznaczną częścią pełnego zbioru.
print("Incomplete rows: ",len(df) - len(df.dropna()))
df = df.dropna()

# Odrzucenie cechy reviewTime, ponieważ oznacza ona to samo co unixReviewTime a jest mniej praktyczna
df.drop("reviewTime", axis=1, inplace=True)

# Kodowanie cech tekstowych na liczbowe
encoders = {}
for col in ['reviewerID','asin','reviewerName']:
    enc = LabelEncoder().fit(df[col])
    df[col] = enc.transform(df[col])
    encoders.update({col:enc})

df.head()

46


,reviewerID,asin,reviewerName,helpful,reviewText,summary,unixReviewTime,score
0,49575,2329,23869,"[0, 0]",I have decided not to play this game. I can't...,Friends,1396396800,1.0
1,75353,11888,50411,"[3, 4]",The Amazon Appstore free app of the day for Ju...,"Amazon Makes This ""Longest Spring Ever"" for Fi...",1402272000,2.0
2,55654,9330,27625,"[0, 4]",this game was so mush fun I wish I could play ...,best,1368921600,5.0
3,76417,4425,6657,"[0, 2]","Its pretty fun and very good looking, but you...",Fun Game,1350172800,5.0
4,58548,12778,25658,"[0, 0]",good graphics; immersive storyline; hard to st...,great game!,1396915200,5.0


Podstawowe informacje o zbiorze

In [49]:
print("---Dataset description---\n", df.describe())
print( "---Correlation Matrix---\n", df.corr())

---Dataset description---
           reviewerID           asin   reviewerName  unixReviewTime  \
count  555745.000000  555745.000000  555745.000000    5.557450e+05   
mean    43623.056312    6102.522288   35618.293870    1.368604e+09   
std     25200.616434    3631.703504   21639.569796    2.353921e+07   
min         0.000000       0.000000       0.000000    1.300752e+09   
25%     21733.000000    2882.000000   16799.000000    1.354579e+09   
50%     43564.000000    6102.000000   35062.000000    1.370650e+09   
75%     65570.000000    9056.000000   54039.000000    1.388707e+09   
max     87222.000000   13207.000000   74191.000000    1.406074e+09   

               score  
count  555745.000000  
mean        3.945410  
std         1.354681  
min         1.000000  
25%         3.000000  
50%         5.000000  
75%         5.000000  
max         5.000000  
---Correlation Matrix---
                 reviewerID      asin  reviewerName  unixReviewTime     score
reviewerID        1.000000 -0.00

In [50]:
# def get_stems(word: str):
#     tokens = nltk.word_tokenize(word)
#
#     porter = nltk.PorterStemmer()
#     for i in range(len(tokens)):
#         tokens[i] = porter.stem(tokens[i])
#
#     return tokens
#
# from collections import Counter
# import re
#
# words = Counter()
# for i in df['reviewText'].index:
#     words.update(get_stems(df['reviewText'][i]))
#
# # words.most_common(50)

Przetwarzanie uzyskanych stemmów.

In [51]:
# for w in list(words):
#     if re.search("\W", w) is not None:
#         del words[w]
# # words.most_common(50)
#
# # nltk.corpus.stopwords.words('english')
# ### Rozważyć wywalanie not
# stopwords = ["a", "about", "after", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been",
#              "before", "being", "between", "both", "by", "could", "did", "do", "does", "doing", "during", "each",
#              "for", "from", "further", "had", "has", "have", "having", "he", "her", "here", "hers", "herself", "him",
#              "himself", "his", "how", "i", "in", "into", "is", "it", "its", "itself", "let", "me", "more", "most", "my",
#              "myself", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "own", "sha",
#              "she", "should", "so", "some", "such", "than", "that", "the", "their", "theirs", "them", "themselves",
#              "then", "there", "there's", "these", "they", "this", "those", "through", "to", "until", "up", "very",
#              "was", "we", "were", "what", "when", "where", "which", "while", "who", "whom", "with", "would", "you",
#              "your", "yours", "yourself", "yourselves",
#              "n't", "'s", "'ll", "'re", "'d", "'m", "'ve",
#              "above", "again", "against", "below", "but", "cannot", "down", "few", "if", "no", "nor", "not", "off",
#              "out", "over", "same", "too", "under", "why"]
#
# for w in stopwords:
#     if w in words:
#         del words[w]
# # words.most_common(50)
#
# ### Usuwanie najrzadszych tokenów.
# max_words = 50
# for i in list(range(len(words)))[max_words:]:
#     del words[i]
# # words.most_common(50)

Tworzenie wektora Bag Of Words

In [52]:
# from scipy.sparse import csr_matrix
#
# #BOW
# # def create_bow(documents, features):
# #     row = []
# #     col = []
# #     data = []
# #
# #     labels = []
# #
# #     for i in documents.index:
# #         tweet_tokens = get_stems(documents[i])
# #
# #         labels.append(label)
# #         for token in set(tweet_tokens):
# #             if token not in features:
# #                 continue
# #             row.append(i)
# #             col.append(features[token])
# #             data.append(1)
# #     return csr_matrix((data, (row, col)), shape=(len(documents), len(features))), labels
#
# min_word_count = 5
#
# common_words = list([k for k, v in words.most_common() if v > min_word_count])
#
# feature_dict = {}
# for word in common_words:
#     feature_dict[word] = len(feature_dict)
#
# # X, y = create_bow(train_tweets, feature_dict)

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=100)

# df['reviewText'] = vectorizer.fit_transform(df['reviewText']).toarray().tolist()
# df['summary'] = vectorizer.fit_transform(df['summary']).toarray().tolist()
df_review = pd.DataFrame(vectorizer.fit_transform(df['reviewText']).toarray())
print(vectorizer.labels)
df_review.head()

# df.head()

AttributeError: 'CountVectorizer' object has no attribute 'labels'

# Uczenie modelu

In [54]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier

y = df["score"]
# X = df[list(set(df.columns) - {"score", 'helpful'})]
X = df_review

print("X columns:",list(X.columns),"\ny columns:", y.name)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print("---Training classifier---")

clf = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=23)
clf.fit(X_train, y_train)

# Tutaj hiperparametry
# grid = GridSearchCV()

X columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99] 
y columns: score
---Training classifier---


RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=23)

Rezultaty klasyfikacji i porównania z algorytmem większościowym oraz losowym.

In [55]:
import random
from sklearn.metrics import f1_score, precision_score, recall_score

list_of_labels = y_train.unique()


def scores(y_true, y_pred, name=None):
    print(f"=================== Results: {name} ===================")
    print(f"            {list_of_labels}   ")
    print("F1       ", f1_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels))
    print("Precision", precision_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels))
    print("Recall   ", recall_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels))


y_pred_random, y_pred_majority = [],[]
majority_label = y_train.value_counts().index[0]

for _ in range(len(X_test)):
    y_pred_random.append(random.choice(list_of_labels))
    y_pred_majority.append(majority_label)

scores(y_test, y_pred_random, "RandomClassifier")
scores(y_test, y_pred_majority, "MajorityClassifier")
scores(y_test, clf.predict(X_test), "Trained Classifier")

=================== Results: RandomClassifier ===================
            [2. 5. 3. 4. 1.]   
F1        [0.09640828 0.28823008 0.14674994 0.2048511  0.14131644]
Precision [0.06285027 0.50788001 0.11529783 0.20998359 0.10915685]
Recall    [0.20685622 0.20120999 0.20179863 0.19996353 0.20034032]
=================== Results: MajorityClassifier ===================
            [2. 5. 3. 4. 1.]   
F1        [0.         0.67243357 0.         0.         0.        ]
Precision [0.         0.50651595 0.         0.         0.        ]


c:\users\festo\onedrive - put.poznan.pl\semestr ix\emd\emd-python\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall    [0. 1. 0. 0. 0.]
=================== Results: Trained Classifier ===================
            [2. 5. 3. 4. 1.]   
F1        [0.01522443 0.72206954 0.18001895 0.2099871  0.42071673]
Precision [0.27619048 0.5922122  0.3428686  0.37199444 0.48276086]
Recall    [0.00782796 0.92487055 0.12204987 0.14628048 0.37280416]
